In [4]:
from splinter import Browser
import time
from bs4 import BeautifulSoup as bs
import pyautogui
import requests
import urllib
import pandas as pd
import re
import datetime
import warnings
warnings.filterwarnings("ignore")

starttime = datetime.datetime.now()

In [5]:
zipcode = 92782
waittime = .25
areatosearch = ''

In [7]:
pagenumber

0

In [8]:
pagelist = []

for pagenumber in range(2):

    clpage = 'https://orangecounty.craigslist.org/search/cta?s=' + str(pagenumber * 120)
    clpage = 'https://orangecounty.craigslist.org/search/cta?s=' + str(pagenumber * 120) + '&auto_paint=1&auto_paint=10&auto_paint=11&auto_paint=2&auto_paint=20&auto_paint=3&auto_paint=4&auto_paint=5&auto_paint=6&auto_paint=7&auto_paint=8&auto_paint=9&bundleDuplicates=1&min_price=1000&postal=92782&search_distance=40'

    clpage = requests.get(clpage)
    clpage = bs(clpage.text, 'html.parser')
    clpage = clpage.find('ul', class_='rows')


    for page in clpage:
        try:
            pagelist.append(page.find('a').attrs['href'])
        except:
            pass

    pagelist = list(set(pagelist))

len(pagelist)

143

In [9]:
# pagelist = pagelist[:25]

In [10]:
cldf = pd.DataFrame()

for page in pagelist:
    
    try:
        attrdict = {}

        attrdict['url'] = page

        page = requests.get(page).text
        page = bs(page, 'html.parser')

        attrgroups = page.find_all('p', class_ ='attrgroup')

        attrdict['clprice'] = page.find('span', class_='price').text.replace('$', '')

        car = attrgroups[0]
        car = car.find('span').text
        attrdict['fullcar'] = car
        attrdict['make'] = car.split(' ', 1)[-1].split(' ', 1)[0]
        attrdict['model'] = car.split(' ', 1)[-1].split(' ', 1)[-1]
        attrdict['year'] = car.split(' ')[0]


        attrs = attrgroups[1].find_all('span')

        for attr in attrs:
            try:
                attr = attr.text
                specname = attr.split(':')[0]
                specvalue = attr.split(': ')[1]
                attrdict[specname] = specvalue
            except:
                continue

        cldf = cldf.append(attrdict, ignore_index = True)
    except Exception as e:
        print(e, attrdict['url'])

('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None)) https://orangecounty.craigslist.org/cto/d/1951-mercury-hot-rod-lead-sled-coupe/6952987363.html


In [11]:
# cldf = cldfextra


In [12]:
cldfextra = cldf
list(cldf.columns)
cldf = cldf[['make', 'model', 'year', 'clprice',  'odometer', 'paint color', 'VIN', 'fullcar', 'url' ]]



In [13]:
nonulls = ['make', 'model', 'year', 'clprice',  'odometer', 'paint color']

for column in nonulls:
    cldf = cldf[cldf[column].notnull()]
    
cldf = cldf.drop_duplicates(keep='first', subset=['VIN', 'fullcar'])


cldf = cldf.reset_index(drop=True)
print(cldf.shape)
cldf.head()

(118, 9)


,make,model,year,clprice,odometer,paint color,VIN,fullcar,url
0,subaru,baja turbo,2005,7700,121876,black,NaN,2005 subaru baja turbo,https://orangecounty.craigslist.org/cto/d/tust...
1,1947,Ford,1947,12000,89620,black,1953749,1947 1947 Ford,https://orangecounty.craigslist.org/cto/d/tust...
2,BMW,X5,2014,20700,82845,black,NaN,2014 BMW X5,https://orangecounty.craigslist.org/ctd/d/tust...
3,Ford,Edge,2013,12559,67330,silver,NaN,2013 Ford Edge,https://losangeles.craigslist.org/lac/ctd/d/tu...
4,BMW,X1,2016,23200,21447,white,NaN,2016 BMW X1,https://losangeles.craigslist.org/lac/ctd/d/tu...


In [14]:
cldf

,make,model,year,clprice,odometer,paint color,VIN,fullcar,url
0,subaru,baja turbo,2005,7700,121876,black,NaN,2005 subaru baja turbo,https://orangecounty.craigslist.org/cto/d/tust...
1,1947,Ford,1947,12000,89620,black,1953749,1947 1947 Ford,https://orangecounty.craigslist.org/cto/d/tust...
2,BMW,X5,2014,20700,82845,black,NaN,2014 BMW X5,https://orangecounty.craigslist.org/ctd/d/tust...
3,Ford,Edge,2013,12559,67330,silver,NaN,2013 Ford Edge,https://losangeles.craigslist.org/lac/ctd/d/tu...
4,BMW,X1,2016,23200,21447,white,NaN,2016 BMW X1,https://losangeles.craigslist.org/lac/ctd/d/tu...
5,1971,vw bus,1971,11900,80000,blue,114638,1971 1971 vw bus,https://orangecounty.craigslist.org/cto/d/tust...
6,nissan,xterra 4x4,2000,3250,193516,silver,5NlED28Y6YC512626,2000 nissan xterra 4x4,https://orangecounty.craigslist.org/cto/d/irvi...
7,chrysler,town and country,2008,4000,137851,silver,2A8HR54PX8R768611,2008 chrysler town and country,https://orangecounty.craigslist.org/cto/d/irvi...
8,Toyota,Pickup,1986,9995,187000,white,NaN,1986 Toyota Pickup,https://orangecounty.craigslist.org/cto/d/1986...
9,acura,mdx,2014,21500,83107,black,NaN,2014 acura mdx,https://losangeles.craigslist.org/lgb/cto/d/tu...


In [15]:
# if make.lower() in ['fiat', 'infiniti']:
#     make = make.upper()
# else:
#     make = make.capitalize()

# if make.lower() == 'vw':
#     make = 'Volkswagen'

# if color == 'Grey':
#                 color = 'Gray'

makedict = {
    'vw': 'Volkswagen',
    'fiat': 'FIAT',
    'infiniti': 'INFINITI',
    'mercedes': 'Mercedes-Benz',
    'gmc': 'GMC',
    'bmw': 'BMW',
}
        
for i, x in cldf.iterrows():
    if x['make'].lower() in list(makedict.keys()):
        cldf.iat[i, cldf.columns.get_loc("make")] = makedict[x['make'].lower()]
        
colordict = {
    'grey': 'gray',
}
        
for i, x in cldf.iterrows():
    if x['paint color'].lower() in list(colordict.keys()):
        cldf.iat[i, cldf.columns.get_loc("paint color")] = colordict[x['paint color'].lower()]

cldf


    

,make,model,year,clprice,odometer,paint color,VIN,fullcar,url
0,subaru,baja turbo,2005,7700,121876,black,NaN,2005 subaru baja turbo,https://orangecounty.craigslist.org/cto/d/tust...
1,1947,Ford,1947,12000,89620,black,1953749,1947 1947 Ford,https://orangecounty.craigslist.org/cto/d/tust...
2,BMW,X5,2014,20700,82845,black,NaN,2014 BMW X5,https://orangecounty.craigslist.org/ctd/d/tust...
3,Ford,Edge,2013,12559,67330,silver,NaN,2013 Ford Edge,https://losangeles.craigslist.org/lac/ctd/d/tu...
4,BMW,X1,2016,23200,21447,white,NaN,2016 BMW X1,https://losangeles.craigslist.org/lac/ctd/d/tu...
5,1971,vw bus,1971,11900,80000,blue,114638,1971 1971 vw bus,https://orangecounty.craigslist.org/cto/d/tust...
6,nissan,xterra 4x4,2000,3250,193516,silver,5NlED28Y6YC512626,2000 nissan xterra 4x4,https://orangecounty.craigslist.org/cto/d/irvi...
7,chrysler,town and country,2008,4000,137851,silver,2A8HR54PX8R768611,2008 chrysler town and country,https://orangecounty.craigslist.org/cto/d/irvi...
8,Toyota,Pickup,1986,9995,187000,white,NaN,1986 Toyota Pickup,https://orangecounty.craigslist.org/cto/d/1986...
9,acura,mdx,2014,21500,83107,black,NaN,2014 acura mdx,https://losangeles.craigslist.org/lgb/cto/d/tu...


In [16]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)


In [ ]:
cldf['kbbprice'] = ''

for j, y in cldf.iterrows():

    for x in range(1):
        try:
            kbbprice = 0

            year = y['year']
            make = y['make']
            model = y['model'].upper()
            mileage = y['odometer']
            color = y['paint color'].capitalize()



            browser.visit('https://www.kbb.com/whats-my-car-worth/')

            browser.find_by_id('yearDropdown').click()
            browser.find_option_by_text(year).first.click()
            browser.find_by_id('yearDropdown').click()

            time.sleep(1 * x + waittime)

            browser.find_by_id('makeDropdown').click()
            try:
                browser.find_option_by_text(make).first.click()
            except:
                try:
                    browser.find_option_by_text(make.capitalize()).first.click()
                except:
                    browser.find_option_by_text(make.upper()).first.click()
                
            browser.find_by_id('makeDropdown').click()

            time.sleep(1 * x + waittime)

            browser.find_by_id('modelDropdown').click()
            
            try:
                browser.find_option_by_text(model).first.click()
            except:
                try:
                    modelwords = re.split(' |-', model)

                    models = bs(browser.html, 'html.parser')
                    models = models.find(id='modelDropdown')
                    models = models.find_all('option')

                    modelcountdict = {}

                    for modeloptions in models:
                        modelcountdict[modeloptions.text] = 0
                        for word in modelwords:
                            if word.lower() in modeloptions.text.lower():
                                modelcountdict[modeloptions.text] += 1
                    chosenmodel = max(modelcountdict, key=modelcountdict.get)
                    
                    browser.find_option_by_text(chosenmodel).first.click()
                    
                except:
                    break
                    
            browser.find_by_id('modelDropdown').click()

            time.sleep(1 * x + waittime)

            browser.fill('mileage', mileage)

            time.sleep(1 * x + waittime)

            browser.fill('zipcode', zipcode)

            time.sleep(1 * x + waittime)

            browser.find_by_id('submitButton').click()

            time.sleep(1 * x + waittime)

            try:
                browser.find_by_text('No thanks').click()
            except:
                pass

            if browser.url == 'https://www.kbb.com/whats-my-car-worth/':
                break
                
                

            if 'styles' in browser.url:
                
                try:
                    browser.find_by_text('No thanks').click()
                except:
                    pass
                
                modelwords = re.split(' |-', model)
                modelwords

                styles = bs(browser.html, 'html.parser')
                styles = styles.find_all('div', class_='style-name-container')

                stylecountdict = {}

                for style in styles:
                    stylecountdict[style.text] = 0
                    for word in modelwords:
                        if word.lower() in style.text.lower():
                            stylecountdict[style.text] += 1

                chosenstyle = max(stylecountdict, key=stylecountdict.get)
                chosenstyle = chosenstyle.split('\n')
                chosenstyle = chosenstyle[1]
                chosenstyle

                browser.execute_script("document.getElementsByClassName('js-style-option')[0].click()")

                browser.execute_script("for (i = 0; i < document.getElementsByClassName('style-radio').length; i++){\
                if(document.getElementsByClassName('style-name')[i].innerText == '" + chosenstyle +"') {\
                document.getElementsByClassName('style-radio')[i].click()\
                    }\
                }")

                time.sleep(2 * (x+1) + waittime)

                browser.find_by_id('stylesNextButton').click()

                if 'styles' in browser.url:
                    break

            browser.execute_script("document.getElementById('standardRadio').click()")

            time.sleep(1 + waittime)

            browser.execute_script("for (i = 0; i < document.getElementsByClassName('option-input').length; i++){\
                        if(document.getElementsByClassName('option-input')[i].innerText.trim() == '" + color + "') {\
                        document.getElementsByClassName('option-input')[i].getElementsByTagName('input')[0].click()\
        }\
    }")

            time.sleep(1 + waittime)

            browser.execute_script("for (i = 0; i < document.getElementsByClassName('title').length; i++){\
                        if(document.getElementsByClassName('title')[i].innerText == 'Good') {\
                        document.getElementsByClassName('title')[i].click()\
        }\
    }")
            
            time.sleep(1 + waittime)

            browser.find_by_id('stylesNextButton').click()

            if 'pricetype' not in browser.url:
                break

            try:
                browser.find_by_text('No thanks').click()
            except:
                pass

            for getprice in range(60):
                try:
#                     browser.execute_script("document.getElementsByClassName('js-private-party-tab')[0].click()")

                    kbbprice = browser.html
                    kbbprice = kbbprice.split('data-privateparty-price="')[-1]
                    kbbprice = kbbprice.split('"')[0]
                    kbbprice = float(kbbprice)
                    
                    print(j, '.  ', kbbprice)
                    
                    cldf.iat[j, cldf.columns.get_loc("kbbprice")] = float(kbbprice)
                    break
                except:
                    time.sleep(.5)
     
                    
            break
        except Exception as e:
            print(e)
            print(f'year = {year}')
            print(f'make = {make}')
            print(f'model = {model}')
            print(f'mileage = {mileage}')
            print(f'color = {color}')
            print(j, browser.url)
            time.sleep(1 * x + waittime)

0 .   7309.0
no elements could be found with option by text "1947"
year = 1947
make = 1947
model = FORD
mileage = 89620
color = Black
1 https://www.kbb.com/whats-my-car-worth/
2 .   19617.0
3 .   10538.0
4 .   25513.0
no elements could be found with option by text "1971"
year = 1971
make = 1971
model = VW BUS
mileage = 80000
color = Blue
5 https://www.kbb.com/whats-my-car-worth/
6 .   1877.0
no elements could be found with option by text "1986"
year = 1986
make = Toyota
model = PICKUP
mileage = 187000
color = White
8 https://www.kbb.com/whats-my-car-worth/
9 .   17878.0
Message: element click intercepted: Element <a id="styles...Button" class="button-one js-path-next pull-right" href="javascript:void(0);">Next</a> is not clickable at point (560, 848). Other element would receive the click: <div id="socialToolbar" class="hideForPrint social-toolbar ">...</div>
  (Session info: chrome=76.0.3809.100)

year = 2011
make = FORD
model = FUSION
mileage = 37000
color = Gray
10 https://www.kbb.c

In [ ]:
cldf.replace('', 0, inplace=True)

cldf = cldf[cldf['kbbprice'] != 0]

cldf['clprice'] = cldf['clprice'].astype(float)
cldf['kbbprice'] = cldf['kbbprice'].astype(float)
cldf['profit'] = cldf['kbbprice'] - cldf['clprice']
cldf = cldf[cldf['profit'] > 0]
cldf['profit/clprice'] = cldf['profit']/cldf['clprice']
cldf = cldf.sort_values(by='profit/clprice', ascending=False)
cldf

try:
    browser.quit()
except:
    pass

In [ ]:
cldf

In [ ]:
cldf.to_csv('Cars.csv')

In [ ]:
print(datetime.datetime.now()-starttime)

In [ ]:
# browser.execute_script(f'document.getElementById("yearDropdown").value = "{year}";')
# browser.find_by_xpath('//option[@value="2015"]').click()
# browser.find_by_xpath('//option[@value="2015"]').click()
# browser.find_by_xpath('//option[@value="2015"]').click()